In [1]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
import argparse
import os
import pprint
import tempfile
import urllib
import zipfile

In [2]:

train = os.path.join('datasets', 'adult/adult.data')
test = os.path.join('datasets', 'adult/adult.test')

In [3]:
import tensorflow_transform as tft
import apache_beam as beam

/Users/ksuchanek/PycharmProjects/weather/venv/lib/python3.7/site-packages/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [4]:
import tensorflow as tf
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
tf.logging.set_verbosity(tf.logging.ERROR)

W0919 16:13:32.891364 4420425152 deprecation_wrapper.py:119] From /Users/ksuchanek/PycharmProjects/weather/venv/lib/python3.7/site-packages/tensorflow_transform/beam/common.py:51: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0919 16:13:32.897917 4420425152 deprecation_wrapper.py:119] From /Users/ksuchanek/PycharmProjects/weather/venv/lib/python3.7/site-packages/tensorflow_transform/beam/impl.py:283: The name tf.SparseTensorValue is deprecated. Please use tf.compat.v1.SparseTensorValue instead.



In [5]:
CATEGORICAL_FEATURE_KEYS = [
    'workclass',
    #'education',
    #'marital-status',
    #'occupation',
    #'relationship',
    #'race',
    #'sex',
    #'native-country',
]
TEXT_HASH_KEYS = [
    'occupation'
]
NUMERIC_FEATURE_KEYS = [
    'age',
    'capital-gain',
    #'capital-loss',
    #'hours-per-week',
]
OPTIONAL_NUMERIC_FEATURE_KEYS = [
    #'education-num',
]
LABEL_KEY = 'label'

In [6]:
RAW_DATA_FEATURE_SPEC = dict(
    [(name, tf.VarLenFeature( tf.string))
     for name in TEXT_HASH_KEYS] +
    [(name, tf.FixedLenFeature([], tf.string))
     for name in CATEGORICAL_FEATURE_KEYS] +
    [(name, tf.FixedLenFeature([], tf.float32))
     for name in NUMERIC_FEATURE_KEYS] +
    [(name, tf.VarLenFeature(tf.float32))
     for name in OPTIONAL_NUMERIC_FEATURE_KEYS] #+
    #[(LABEL_KEY, tf.FixedLenFeature([], tf.string))]
)

RAW_DATA_METADATA = dataset_metadata.DatasetMetadata(
    dataset_schema.from_feature_spec(RAW_DATA_FEATURE_SPEC))

In [7]:
RAW_DATA_FEATURE_SPEC

{'occupation': VarLenFeature(dtype=tf.string),
 'workclass': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 'age': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'capital-gain': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None)}

In [8]:
testing = True
if testing:
  TRAIN_NUM_EPOCHS = 1
  NUM_TRAIN_INSTANCES = 1
  TRAIN_BATCH_SIZE = 1
  NUM_TEST_INSTANCES = 1
else:
  TRAIN_NUM_EPOCHS = 16
  NUM_TRAIN_INSTANCES = 32561
  TRAIN_BATCH_SIZE = 128
  NUM_TEST_INSTANCES = 16281

# Names of temp files
TRANSFORMED_TRAIN_DATA_FILEBASE = 'train_transformed'
TRANSFORMED_TEST_DATA_FILEBASE = 'test_transformed'
EXPORTED_MODEL_DIR = 'exported_model_dir'

In [9]:
class MapAndFilterErrors(beam.PTransform):
  """Like beam.Map but filters out erros in the map_fn."""

  class _MapAndFilterErrorsDoFn(beam.DoFn):
    """Count the bad examples using a beam metric."""

    def __init__(self, fn):
      self._fn = fn
      # Create a counter to measure number of bad elements.
      self._bad_elements_counter = beam.metrics.Metrics.counter(
          'census_example', 'bad_elements')

    def process(self, element):
      try:
        yield self._fn(element)
      except Exception:  # pylint: disable=broad-except
        # Catch any exception the above call.
        self._bad_elements_counter.inc(1)

  def __init__(self, fn):
    self._fn = fn

  def expand(self, pcoll):
    return pcoll | beam.ParDo(self._MapAndFilterErrorsDoFn(self._fn))

In [10]:
def preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    # Since we are modifying some features and leaving others unchanged, we
    # start by setting `outputs` to a copy of `inputs.
    outputs = inputs.copy()
    res = dict()
    # Scale numeric columns to have range [0, 1].
    for key in NUMERIC_FEATURE_KEYS:
        res[key] = tft.scale_to_0_1(outputs[key], name=key)

    for key in OPTIONAL_NUMERIC_FEATURE_KEYS:
    # This is a SparseTensor because it is optional. Here we fill in a default
    # value when it is missing.
        dense = tf.sparse_to_dense(outputs[key].indices,
                                   [outputs[key].dense_shape[0], 1],
                                   outputs[key].values, default_value=0.)
        # Reshaping from a batch of vectors of size 1 to a batch to scalars.
        dense = tf.squeeze(dense, axis=1)
        res[key] = tft.scale_to_0_1(dense, name = key)
        
    for key in TEXT_HASH_KEYS:
        dense = tf.sparse_to_dense(outputs[key].indices,
                                   [outputs[key].dense_shape[0], 1],
                                   outputs[key].values, default_value="", name = key)
        tmp = tft.hash_strings(dense,
            hash_buckets=1024,
            name="hash_" + key
        )
        dense = tf.squeeze(tmp, axis=1)
        res[key]  = tf.cast(dense,tf.float32)


    # For all categorical columns except the label column, we generate a
    # vocabulary but do not modify the feature.  This vocabulary is instead
    # used in the trainer, by means of a feature column, to convert the feature
    # from a string to an integer id.
    for key in CATEGORICAL_FEATURE_KEYS:
        v1 = tft.compute_and_apply_vocabulary(
            inputs[key],
            default_value=-1,
            top_k=100,
            frequency_threshold=0.05,
            num_oov_buckets=0,
            vocab_filename=key,
            weights=None,
            labels=None,
            use_adjusted_mutual_info=False,
            min_diff_from_avg=0.0,
            coverage_top_k=None,
            coverage_frequency_threshold=None,
            key_fn=None,

            name=key
        )
        
        res[key] = tf.cast(v1,tf.float32)
        #that code modifies some global, shared state
     
    # For the label column we provide the mapping from string to index.
    #table = tf.contrib.lookup.index_table_from_tensor(['>50K', '<=50K'])
    #outputs[LABEL_KEY] = table.lookup(outputs[LABEL_KEY], name=LABEL_KEY)
    # outputs['XXALLXX'] = 

    return {"XXALLXX":tf.stack([res[o] for o in res],axis=1, name="output_all")}

In [11]:
def transform_data(train_data_file, test_data_file, working_dir):
  """Transform the data and write out as a TFRecord of Example protos.

  Read in the data using the CSV reader, and transform it using a
  preprocessing pipeline that scales numeric data and converts categorical data
  from strings to int64 values indices, by creating a vocabulary for each
  category.

  Args:
    train_data_file: File containing training data
    test_data_file: File containing test data
    working_dir: Directory to write transformed data and metadata to
  """

  # The "with" block will create a pipeline, and run that pipeline at the exit
  # of the block.
  with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
      # Create a coder to read the census data with the schema.  To do this we
      # need to list all columns in order since the schema doesn't specify the
      # order of columns in the csv.
      ordered_columns = [
          'age', 'workclass', 'fnlwgt', 'education', 'education-num',
          'marital-status', 'occupation', 'relationship', 'race', 'sex',
          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
          'label'
      ]
      converter = tft.coders.CsvCoder(ordered_columns, RAW_DATA_METADATA.schema)

      # Read in raw data and convert using CSV converter.  Note that we apply
      # some Beam transformations here, which will not be encoded in the TF
      # graph since we don't do the from within tf.Transform's methods
      # (AnalyzeDataset, TransformDataset etc.).  These transformations are just
      # to get data into a format that the CSV converter can read, in particular
      # removing spaces after commas.
      #
      # We use MapAndFilterErrors instead of Map to filter out decode errors in
      # convert.decode which should only occur for the trailing blank line.
      raw_data = (
          pipeline
          | 'ReadTrainData' >> beam.io.ReadFromText(train_data_file)
          | 'FixCommasTrainData' >> beam.Map(
              lambda line: line.replace(', ', ','))
          | 'DecodeTrainData' >> MapAndFilterErrors(converter.decode))

      # Combine data and schema into a dataset tuple.  Note that we already used
      # the schema to read the CSV data, but we also need it to interpret
      # raw_data.
      raw_dataset = (raw_data, RAW_DATA_METADATA)
      #transformed_dataset, transform_fn = (
      #    raw_dataset | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn))
          
      transform_fn =  raw_dataset |  tft_beam.AnalyzeDataset(preprocessing_fn)

      print("workdir",working_dir )
      #print(transform_fn)
      _ = (
          transform_fn
          | 'WriteTransformFn' >> tft_beam.WriteTransformFn(working_dir))
      return transform_fn
        

In [12]:
tf.reset_default_graph()
transform_data(train, test, "/tmp/tft05")

workdir /tmp/tft05


(<PCollection[AnalyzeDataset/CreateSavedModel/BindTensors.None] at 0x1359e5a90>,
 BeamDatasetMetadata(dataset_metadata={'_schema': Schema(feature {
   name: "XXALLXX"
   type: FLOAT
   presence {
     min_fraction: 1.0
   }
   shape {
     dim {
       size: 4
     }
   }
 }
 )}, deferred_metadata=<PCollection[AnalyzeDataset/ComputeDeferredMetadata.None] at 0x1359f4978>))

In [27]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile

from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.Session() as sess:
    model_filename ='/tmp/tft05/transform_fn/saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:

        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)

        if 1 != len(sm.meta_graphs):
            print('More than one graph found. Not sure which to write')
            sys.exit(1)

      
        g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
LOGDIR='/tmp/logs05'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

In [21]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile
tf.reset_default_graph()
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat
import tensorflow.contrib as tfc
with tf.Session() as sess:
    model_filename ='/tmp/tft05/transform_fn/saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:

        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)

        if 1 != len(sm.meta_graphs):
            print('More than one graph found. Not sure which to write')
            sys.exit(1)

        
        g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
       
    
        opName = "import/transform/workclass/apply_vocab/string_to_index/hash_table/table_init/InitializeTableFromTextFileV2"
        sess.run( sess.graph.get_operation_by_name(opName))
        #sess.run(tf.initializers.tables_initializer(name='initialize_all_tables'))
        in1 = sess.graph.get_tensor_by_name("import/transform/inputs/age:0")
      
        out1 = sess.graph.get_tensor_by_name("import/transform/output_all:0")
        
        in2 = sess.graph.get_tensor_by_name("import/transform/inputs/capital-gain:0")
        in3 = sess.graph.get_tensor_by_name("import/transform/inputs/workclass:0")
        for o in sess.graph.get_operations():
            print(o)
        in4s = sess.graph.get_tensor_by_name("import/transform/inputs/occupation/shape:0")
        in4v = sess.graph.get_tensor_by_name("import/transform/inputs/occupation/values:0")
        in4i = sess.graph.get_tensor_by_name("import/transform/inputs/occupation/indices:0")


      
        #print(op)
        for o in sess.graph.get_operations():
            print(o)
        res = sess.run([out1], feed_dict = {in1: [35,60], in2:[100000, 80000], in3:["Local-gov","bbbb"],
                                            in4s:[1, 1], in4v:["a","b"],
                                            in4i:[1,1]})
        print(res)
        print(out1)

name: "import/Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_STRING
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_STRING
      tensor_shape {
      }
      string_val: "/var/folders/h3/zt26x1d93hq49vff996w04pc0000gq/T/tmportp5w4g/tftransform_tmp/workclass"
    }
  }
}

name: "import/Const_1"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 99999.0
    }
  }
}

name: "import/Const_2"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 0.0
    }
  }
}

name: "import/Const_3"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: -17.0
    }
  }
}

name: "import/Const_

ValueError: Cannot feed value of shape (2,) for Tensor 'import/transform/inputs/occupation/indices:0', which has shape '(?, 2)'

In [ ]:
from tensorflow.graph_util import extract_sub_graph

![graph](graph_scale_0_1.png)